In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('seaborn-notebook')
%matplotlib inline

In [38]:
df = pd.read_csv('../data/data_zipcode_imported.csv')

In [39]:
# SF business locations file
df_b = pd.read_csv('../data/Registered_Business_Locations_-_San_Francisco.csv')

In [40]:
df_b_3cols = df_b[['Street Address', 'Source Zipcode', 'Business Start Date']]

In [41]:
# let's identify '1051 Market St' business transactions in SF business loc
df_b_3cols[df_b_3cols['Street Address'] == '1051 Market St']

,Street Address,Source Zipcode,Business Start Date
65775,1051 Market St,94103.0,02/01/2008
135617,1051 Market St,94103.0,12/17/2014
214983,1051 Market St,94103.0,09/01/2013


In [42]:
# Let's get all the addresses
address_lst = df['business_address'].tolist()

In [43]:
len(address_lst)

5229

In [44]:
address_unique_lst = list(set(address_lst))

In [45]:
len(address_unique_lst)

4829

In [46]:
df_otg = df[df['business_address'] == 'Off The Grid']

In [47]:
len(df_otg)

41

In [48]:
# No address or unreal addresses will be assigned with the average startdate.

In [49]:
for address in address_unique_lst:
    df_b_3unique = df_b_3cols[df_b_3cols['Street Address'] == address]
    if len(df_b_3unique) > 0:
        num_turnovers = len(df_b_3unique)
        latest_startdate = pd.to_datetime(max(df_b_3unique['Business Start Date'].values))
        # let's append these info onto df
        idx = df[df['business_address'] == address].index
        df.loc[idx,'number_turnovers'] = num_turnovers
        df.loc[idx,'start_date'] = latest_startdate

In [18]:
df_test = df_b_3cols[df_b_3cols['Street Address'] == '1051 Market St']

In [19]:
df_test

,Street Address,Source Zipcode,Business Start Date
65775,1051 Market St,94103.0,02/01/2008
135617,1051 Market St,94103.0,12/17/2014
214983,1051 Market St,94103.0,09/01/2013


In [20]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date
0,0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,...,2017-09-28,0,0,5,0,0,6,5,1.0,2003-02-28
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,2016-12-06,0,6,0,0,5,5,3,NaN,NaT


In [21]:
df[df['business_address'] == '1051 Market St']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date
85,85,85,115,81509,The Flying Falafel,1051 Market St,San Francisco,CA,94103,NaN,...,2016-12-05,5,0,0,0,4,0,4,3.0,2014-12-17


### Let's input the average value to missing number_trunovers and start_dates

In [22]:
df[df['number_turnovers'].isnull()].head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,2016-12-06,0,6,0,0,5,5,3,NaN,NaT
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,2017-12-07,0,5,0,0,6,0,0,NaN,NaT


In [23]:
mask_turnovers = df['number_turnovers'].isnull()

In [24]:
# sum(~mask_turnovers) is the number of True in number_turnovers
avg_turnover = sum(df['number_turnovers'][~mask_turnovers])/sum(~mask_turnovers)

In [25]:
df[df['start_date'].isnull()].head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,2016-12-06,0,6,0,0,5,5,3,NaN,NaT
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,2017-12-07,0,5,0,0,6,0,0,NaN,NaT


In [26]:
df.loc[mask_turnovers, 'number_turnovers'] = avg_turnover

In [27]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,inspect_date,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date
0,0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,...,2017-09-28,0,0,5,0,0,6,5,1.000000,2003-02-28
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,2016-12-06,0,6,0,0,5,5,3,4.447019,NaT


## convert datetime in start_date to integer to use it for modeling

In [50]:
#df.loc[:,'start_date'] = df.loc[:,'start_date'].dt.strftime('%Y%m%d')

In [30]:
#df.loc[~mask_turnovers,'start_date'] = df.loc[~mask_turnovers,'start_date'].astype(int)

In [31]:
#avg_startdate = sum(df['start_date'][~mask_turnovers].values)/sum(~mask_turnovers)

In [32]:
#df.loc[mask_turnovers, 'start_date'] = int(avg_startdate)

In [51]:
#df.head(5)

## Years in business 

In [ ]:
# Find the months in business for each restaurant by subtracting their starting date from End of March 2018.

In [52]:
df['start_date'].head()

0   2003-02-28
1          NaT
2   2011-06-15
3          NaT
4   2010-04-01
Name: start_date, dtype: datetime64[ns]

In [53]:
end_of_march_2018 = latest_startdate = pd.to_datetime('03/31/2018')

In [54]:
end_of_march_2018

Timestamp('2018-03-31 00:00:00')

In [96]:
df['duration_business'] = ((end_of_march_2018 - df['start_date'])/ np.timedelta64(1, 'M'))

In [104]:
((end_of_march_2018 - df['start_date']))

0       5510 days
1             NaT
2       2481 days
3             NaT
4       2921 days
5       2190 days
6      12478 days
7      12448 days
8       3379 days
9             NaT
10      4291 days
11      1209 days
12      4644 days
13            NaT
14            NaT
15            NaT
16      2720 days
17            NaT
18            NaT
19      2094 days
20      6512 days
21            NaT
22      7102 days
23      8198 days
24     10375 days
25      2389 days
26            NaT
27            NaT
28      2403 days
29      4784 days
          ...    
5199    3463 days
5200    4168 days
5201     620 days
5202    3742 days
5203          NaT
5204          NaT
5205    6390 days
5206          NaT
5207    3120 days
5208          NaT
5209          NaT
5210          NaT
5211          NaT
5212    5295 days
5213          NaT
5214          NaT
5215          NaT
5216    6358 days
5217          NaT
5218    5456 days
5219          NaT
5220          NaT
5221          NaT
5222     134 days
5223     1

In [90]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date,duration_business
0,0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,...,0,0,5,0,0,6,5,1.0,2003-02-28,181.030411
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,0,6,0,0,5,5,3,NaN,NaT,NaN
2,2,2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,...,4,0,2,0,3,3,2,3.0,2011-06-15,81.512967
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,0,5,0,0,6,0,0,NaN,NaT,NaN
4,4,4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,...,0,11,0,0,0,6,6,2.0,2010-04-01,95.969116


In [91]:
avg_duration = sum(df['duration_business'][~mask_turnovers].values)/sum(~mask_turnovers)

In [92]:
df.loc[mask_turnovers, 'duration_business'] = avg_duration

In [93]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,...,p1_3,p4_6,p7_9,p10_12,p13_18,p19_24,p25_36,number_turnovers,start_date,duration_business
0,0,0,0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,...,0,0,5,0,0,6,5,1.0,2003-02-28,181.030411
1,1,1,1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,...,0,6,0,0,5,5,3,NaN,NaT,153.101342
2,2,2,2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,...,4,0,2,0,3,3,2,3.0,2011-06-15,81.512967
3,3,3,3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,...,0,5,0,0,6,0,0,NaN,NaT,153.101342
4,4,4,4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,...,0,11,0,0,0,6,6,2.0,2010-04-01,95.969116
